In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
name = 'Kaggle User'
email = 'your.email@gmail.com'

url = 'http://ufcstats.com/statistics/events/completed'
headers = {
    'User-Agent' name,
    'From': email
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

with open('ufc_stats.html', 'w') as f:
    f.write(soup.prettify())




print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   Stats | UFC
  </title>
  <meta content="" name="description"/>
  <meta content="" name="viewport"/>
  <link href="/blocks/main.css?ver=537364" rel="stylesheet"/>
  <script src="/js/vendor/modernizr-2.6.2.min.js">
  </script>
  <script>
   (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
    })(window,document,'script','//www.google-analytics.com/analytics.js','ga');

    ga('create', 'UA-2855164-1', 'au

In [ ]:
# @title Fight Details Example
url_ = 'http://ufcstats.com/fight-details/1d68bb733de24b06'
headers = {
    'User-Agent':'Alex Kamp',
    'From':'ak9023@stern.nyu.edu'
}

response = requests.get(url_, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
"""
details_find = soup.find('div', class_='b-fight-details__content')
details = details_find.get_text(strip=True)
details
"""

details_find = soup.find('div', class_='b-fight-details__content')
details_text = details_find.get_text(separator="\n").strip()

# Clean up the output
lines = [line.strip() for line in details_text.splitlines() if line.strip()]

# Initialize an empty dictionary
details_dict = {}

# Parse the cleaned lines to populate the dictionary
current_key = None
for line in lines:
    if ":" in line:  # This line is a key
        current_key = line.replace(":", "").strip()
        details_dict[current_key] = ""
    elif current_key:  # This line is a value for the last key
        details_dict[current_key] += (line.strip() + " ")

# Remove trailing whitespace from each value
details_dict = {k: v.strip() for k, v in details_dict.items()}


print(details_dict)

{'Method': 'Decision - Split', 'Round': '5', 'Time': '', '500': '', 'Time format': '5 Rnd (5-5-5-5-5)', 'Referee': 'Jason Herzog', 'Details': "Sal D'amato 47 - 48. Derek Cleary 48 - 47. Mike Bell 47 - 48."}


In [ ]:
# @title Events Scraper (Model)
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL for UFC events pages
base_url = 'http://ufcstats.com/statistics/events/completed?page='

# List to store fight methods
fight_methods = []

def scrape_event_links(page_number):
    url = base_url + str(page_number)
    headers = {
        'User-Agent': "Kaggle User",
        'From': 'user@gmail.com'
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    event_links = []

    # Find the table containing events
    table = soup.find('table', class_='b-statistics__table-events')

    ##### EVENT LINKS SCRAPER ######
    event_links = []
    # Page Number 1
    if page_number == 1:
      if table:
          tbody = table.find('tbody')
          # Iterate over each row (event) in the table
          ##### IMPORTANT: THIS SKIPS THE FIRST EVENT OF EACH PAGE, SCRAPE Seperately
          for row in tbody.find_all('tr')[2:]:
              link_tag = row.find('a')
              if link_tag:
                  event_url = link_tag.get('href')
                  event_links.append(event_url)

    else:

      if table:
          tbody = table.find('tbody')
          # Iterate over each row (event) in the table
          ##### IMPORTANT: THIS SKIPS THE FIRST EVENT OF EACH PAGE, SCRAPE Seperately
          for row in tbody.find_all('tr')[1:]:
              link_tag = row.find('a')
              if link_tag:
                  event_url = link_tag.get('href')
                  event_links.append(event_url)


    return event_links



# Scrape event details
def scrape_event_details(page_number):

    url_events = 'http://ufcstats.com/statistics/events/completed?page=' + str(page_number)
    headers = {
        'User-Agent': "Kaggle User",
        'From': 'user@gmail.com'
    }

    response = requests.get(url_events, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find('table', class_='b-statistics__table-events')

    event_details = []

    if table:
          tbody = table.find('tbody')
          # Iterate over each row (event) in the table
          ##### IMPORTANT: THIS SKIPS THE FIRST EVENT OF EACH PAGE, SCRAPE Seperately
          for row in tbody.find_all('tr')[1:]:
              event = row.find('a')
              if event:
                event_mini = []

                date = row.find('span', class_='b-statistics__date').get_text(strip=True)
                location = row.find('td', class_='b-statistics__table-col b-statistics__table-col_style_big-top-padding').get_text(strip=True)
                event_title = event.get_text(strip=True)

                event_mini.append(event_title)
                event_mini.append(date)
                event_mini.append(location)

                event_details.append(event_mini)



    return event_details



In [ ]:
# @title Instance Object Event Links and Details (Model)
event_url_list=[]
event_details_list = []

for page in range(1,15):

  event_links = scrape_event_links(str(page))
  details_list = scrape_event_details(str(page))

  for i in details_list:
      event_details_list.append(i)

  for link in event_links:
      event_url_list.append(link)

event_url_list = event_url_list[1:]

print(event_url_list)
print(event_details_list)

df_event_details = pd.DataFrame(event_details_list, columns=['Event', 'Date', 'Location'])

['http://ufcstats.com/event-details/585f9ffdb0cd0466', 'http://ufcstats.com/event-details/3ab0df1ddd5f1dc2', 'http://ufcstats.com/event-details/221b2a3070c7ce3e', 'http://ufcstats.com/event-details/6a0047ac8c99fece', 'http://ufcstats.com/event-details/5bdcbd8dd681a257', 'http://ufcstats.com/event-details/eee8efec7b951d84', 'http://ufcstats.com/event-details/079488710f012779', 'http://ufcstats.com/event-details/0c01568b1ac77bff', 'http://ufcstats.com/event-details/ee0b69e307c857e5', 'http://ufcstats.com/event-details/be8ad887e4d674b0', 'http://ufcstats.com/event-details/32ba186f4b0e3267', 'http://ufcstats.com/event-details/2c733e059462c1dc', 'http://ufcstats.com/event-details/fe435e440b5de0ff', 'http://ufcstats.com/event-details/026b4f7049085842', 'http://ufcstats.com/event-details/c11036da9162cb5f', 'http://ufcstats.com/event-details/66e0a70352fef46a', 'http://ufcstats.com/event-details/f3743d8ef5dde970', 'http://ufcstats.com/event-details/b12a1d19ee383b7c', 'http://ufcstats.com/event-

In [ ]:
# @title Fight Scrape link (Model)
def scrape_fight_links(event_link):
    headers = {
        'User-Agent': "Alex Kamp",
        'From': 'ak9023@stern.nyu.edu'
    }

    response = requests.get(event_link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    fight_links = []
    winner_list = []

    # Find the table containing fight details
    table = soup.find('table')


    if table:
        tbody = soup.find('tbody')
        for row in tbody.find_all('tr'):
            td_tags = row.find_all('td')  # Find all <td> elements in the row
            if len(td_tags) >= 2:  # Ensure there are at least 2 <td> tags
                second_td = td_tags[1]  # Access the second <td> (index 1)

                p_tags = second_td.find_all('p')  # Find all <p> tags in the second <td>
                if len(p_tags) >= 1:  # Ensure there's at least one <p> tag
                    winner = p_tags[0].get_text(strip=True)  # Get the text of the first <p> in the second <td>
                    winner_list.append(winner)

            # Get the fight link (still within the <tr>)
            fight_lin = row.find('a').get('href')
            fight_links.append(fight_lin)

    return fight_links, winner_list



fights = []

for i in event_url_list:
  fights.append(scrape_fight_links(i))




fight_urls = [url for event in fights for url in event[0]]

# Output the list of fight URLs
print(fight_urls)


['http://ufcstats.com/fight-details/73f451d894a1d4ca', 'http://ufcstats.com/fight-details/154138c8fc89c315', 'http://ufcstats.com/fight-details/e6685759ed2af90d', 'http://ufcstats.com/fight-details/c31d9cac7534d879', 'http://ufcstats.com/fight-details/ea6427379a83bd01', 'http://ufcstats.com/fight-details/e62033cb1e79aa81', 'http://ufcstats.com/fight-details/c9d2a718f39274c0', 'http://ufcstats.com/fight-details/6b9ed3867dd42b3b', 'http://ufcstats.com/fight-details/6fd1edb58d271f7c', 'http://ufcstats.com/fight-details/582846744e379178', 'http://ufcstats.com/fight-details/32f2965b4f4c4ebf', 'http://ufcstats.com/fight-details/799acf3ae8a5df0a', 'http://ufcstats.com/fight-details/18729b11614406bf', 'http://ufcstats.com/fight-details/7edf2fe3a8776a44', 'http://ufcstats.com/fight-details/eaa885cf7ae31e0b', 'http://ufcstats.com/fight-details/6ad594777cce2d7d', 'http://ufcstats.com/fight-details/d29a82322bb083d9', 'http://ufcstats.com/fight-details/b989d4d7e220b3c1', 'http://ufcstats.com/fight-

In [ ]:
# @title Judges Scorecards Scraping (Details)

import requests
from bs4 import BeautifulSoup
import pandas as pd

class UFCFightMethodScraper:
    def __init__(self, url):
        """
        Initializes the scraper with the fight URL.
        """
        self.url = url
        self.soup = None

    def load_content(self):
        """
        Loads the HTML content from the fight URL.
        """
        response = requests.get(self.url)
        if response.status_code == 200:
            self.soup = BeautifulSoup(response.content, 'html.parser')
        else:
            print(f"Failed to load content from {self.url}, status code: {response.status_code}")

    def extract_method_of_finish(self):
        """
        Extracts the method of finish (KO/TKO, Submission, Decision) and judges' scorecards if applicable.
        """
        method_tag = self.soup.find('i', class_='b-fight-details__text-item_first')  # For the method of finish
        method_of_finish = method_tag.get_text(strip=True) if method_tag else 'N/A'

        return method_of_finish

    def extract_details(self):
        """
        Extracts details (scorecard or method of finish) from fight

        details_find = soup.find('div', class_='b-fight-details__content')
        details = details_find.get_text(strip=True)
        details
        """

        details_find = self.soup.find('div', class_='b-fight-details__content')
        details_text = details_find.get_text(separator="\n").strip()

        # Clean up the output
        lines = [line.strip() for line in details_text.splitlines() if line.strip()]

        # Initialize an empty dictionary
        details_dict = {}

        # Parse the cleaned lines to populate the dictionary
        current_key = None
        for line in lines:
            if ":" in line:  # This line is a key
                current_key = line.replace(":", "").strip()
                details_dict[current_key] = ""
            elif current_key:  # This line is a value for the last key
                details_dict[current_key] += (line.strip() + " ")

        # Remove trailing whitespace from each value
        details_dict = {k: v.strip() for k, v in details_dict.items()}

        return details_dict


# Function to scrape multiple fight URLs
def scrape_fight_methods(fight_urls):
    results = []
    for url in fight_urls:
        scraper = UFCFightMethodScraper(url)
        scraper.load_content()

        if scraper.soup:
            method_of_finish = scraper.extract_method_of_finish()
            extract_details = scraper.extract_details()

            results.append({
                'Method of Finish': method_of_finish,
                'Details': extract_details,
            })
        else:
            print(f"Failed to load content for URL: {url}")

    return pd.DataFrame(results)

# Example usage


df_fight_methods = scrape_fight_methods(fight_urls)

# Display the DataFrame
print(df_fight_methods)

# Optionally, save the DataFrame to CSV
df_fight_methods.to_csv("fight_methods_and_judges_scores.csv", index=False)

                 Method of Finish  \
0                   Method:KO/TKO   
1               Method:Submission   
2     Method:Decision - Unanimous   
3     Method:Decision - Unanimous   
4                   Method:KO/TKO   
...                           ...   
4161            Method:Submission   
4162                Method:KO/TKO   
4163  Method:Decision - Unanimous   
4164            Method:Submission   
4165                Method:KO/TKO   

                                                Details  
0     {'Method': 'KO/TKO', 'Round': '1', 'Time': '',...  
1     {'Method': 'Submission', 'Round': '3', 'Time':...  
2     {'Method': 'Decision - Unanimous', 'Round': '3...  
3     {'Method': 'Decision - Unanimous', 'Round': '3...  
4     {'Method': 'KO/TKO', 'Round': '1', 'Time': '',...  
...                                                 ...  
4161  {'Method': 'Submission', 'Round': '1', 'Time':...  
4162  {'Method': 'KO/TKO', 'Round': '1', 'Time': '',...  
4163  {'Method': 'Decision - Unan

In [ ]:
# @title Details Scraping vII (Model)

import requests
from bs4 import BeautifulSoup
import pandas as pd

class UFCFightMethodScraper:
    def __init__(self, url):
        """
        Initializes the scraper with the fight URL.
        """
        self.url = url
        self.soup = None

    def load_content(self):
        """
        Loads the HTML content from the fight URL.
        """
        response = requests.get(self.url)
        if response.status_code == 200:
            self.soup = BeautifulSoup(response.content, 'html.parser')
        else:
            print(f"Failed to load content from {self.url}, status code: {response.status_code}")

    def extract_method_of_finish(self):
        """
        Extracts the method of finish (KO/TKO, Submission, Decision).
        """
        method_tag = self.soup.find('i', class_='b-fight-details__text-item_first')
        method_of_finish = method_tag.get_text(strip=True) if method_tag else 'N/A'
        return method_of_finish

    def extract_details(self):
        """
        Extracts detailed fight information including Method, Round, Time, Referee, and Additional Details.
        """
        details_find = self.soup.find('div', class_='b-fight-details__content')
        if not details_find:
            return {}

        details_text = details_find.get_text(separator="\n").strip()
        lines = [line.strip() for line in details_text.splitlines() if line.strip()]

        details_dict = {}
        current_key = None
        for line in lines:
            if ":" in line:  # Line is a key
                current_key = line.replace(":", "").strip()
                details_dict[current_key] = ""
            elif current_key:  # Line is a value for the last key
                details_dict[current_key] += (line.strip() + " ")

        details_dict = {k: v.strip() for k, v in details_dict.items()}

        # Extract additional fight title and specific detail text
        fight_title_tag = self.soup.find('i', class_='b-fight-details__fight-title')
        details_tag = details_find.find_next("p", class_="b-fight-details__text")

        if fight_title_tag:
            details_dict["Title"] = fight_title_tag.get_text(strip=True)

        if details_tag:
            details_dict["Additional Details"] = details_tag.get_text(strip=True)

        return details_dict


# Function to scrape multiple fight URLs
def scrape_fight_methods(fight_urls):
    results = []
    for url in fight_urls:
        scraper = UFCFightMethodScraper(url)
        scraper.load_content()

        if scraper.soup:
            method_of_finish = scraper.extract_method_of_finish()
            details = scraper.extract_details()

            results.append({
                'Method of Finish': method_of_finish,
                'Details': details,
            })
        else:
            print(f"Failed to load content for URL: {url}")

    return pd.DataFrame(results)


df_fight_methods = scrape_fight_methods(fight_urls)

# Display the DataFrame
print(df_fight_methods)

# Optionally, save the DataFrame to CSV
df_fight_methods.to_csv("fight_methods_and_judges_scores.csv", index=False)

                 Method of Finish  \
0                   Method:KO/TKO   
1               Method:Submission   
2     Method:Decision - Unanimous   
3     Method:Decision - Unanimous   
4                   Method:KO/TKO   
...                           ...   
4161            Method:Submission   
4162                Method:KO/TKO   
4163  Method:Decision - Unanimous   
4164            Method:Submission   
4165                Method:KO/TKO   

                                                Details  
0     {'Method': 'KO/TKO', 'Round': '1', 'Time': '',...  
1     {'Method': 'Submission', 'Round': '3', 'Time':...  
2     {'Method': 'Decision - Unanimous', 'Round': '3...  
3     {'Method': 'Decision - Unanimous', 'Round': '3...  
4     {'Method': 'KO/TKO', 'Round': '1', 'Time': '',...  
...                                                 ...  
4161  {'Method': 'Submission', 'Round': '1', 'Time':...  
4162  {'Method': 'KO/TKO', 'Round': '1', 'Time': '',...  
4163  {'Method': 'Decision - Unan

In [ ]:
# @title Details Scraping (Model + Event)
import requests
from bs4 import BeautifulSoup
import pandas as pd

class UFCFightMethodScraper:
    def __init__(self, url):
        """
        Initializes the scraper with the fight URL.
        """
        self.url = url
        self.soup = None

    def load_content(self):
        """
        Loads the HTML content from the fight URL.
        """
        response = requests.get(self.url)
        if response.status_code == 200:
            self.soup = BeautifulSoup(response.content, 'html.parser')
        else:
            print(f"Failed to load content from {self.url}, status code: {response.status_code}")

    def extract_event_name(self):
        """
        Extracts the event name, e.g., "UFC 308: Topuria vs. Holloway".
        """
        event_tag = self.soup.find('h2', class_='b-content__title')
        event_name = event_tag.get_text(strip=True) if event_tag else 'N/A'
        return event_name

    def extract_title_bout(self):
        """
        Extracts the title bout information, e.g., "UFC Featherweight Title Bout".
        """
        title_bout_tag = self.soup.find('i', class_='b-fight-details__fight-title')
        title_bout = title_bout_tag.get_text(strip=True) if title_bout_tag else 'N/A'
        return title_bout

    def extract_method_of_finish(self):
        """
        Extracts the method of finish (KO/TKO, Submission, Decision).
        """
        method_tag = self.soup.find('i', class_='b-fight-details__text-item_first')
        method_of_finish = method_tag.get_text(strip=True) if method_tag else 'N/A'
        return method_of_finish

    def extract_details(self):
        """
        Extracts detailed fight information (Method, Round, Time, Time format, Referee, etc.).
        """
        details_find = self.soup.find('div', class_='b-fight-details__content')
        if not details_find:
            return {}  # Return an empty dictionary if no details are found

        details_text = details_find.get_text(separator="\n").strip()
        lines = [line.strip() for line in details_text.splitlines() if line.strip()]

        details_dict = {}
        current_key = None
        for line in lines:
            if ":" in line:  # Line is a key
                current_key = line.replace(":", "").strip()
                details_dict[current_key] = ""
            elif current_key:  # Line is a value for the last key
                details_dict[current_key] += (line.strip() + " ")

        details_dict = {k: v.strip() for k, v in details_dict.items()}
        return details_dict

    def extract_specific_fight_details(self):
        """
        Extracts specific fight details, e.g., "Punch to Head At Distance".
        """
        details_tag = self.soup.find_all('p', class_='b-fight-details__text')
        fight_details = "N/A"
        for tag in details_tag:
            if "Details:" in tag.get_text():
                fight_details = tag.get_text(strip=True).replace("Details:", "").strip()
                break
        return fight_details


# Function to scrape multiple fight URLs
def scrape_fight_methods(fight_urls):
    results = []
    for url in fight_urls:
        scraper = UFCFightMethodScraper(url)
        scraper.load_content()

        if scraper.soup:
            event_name = scraper.extract_event_name()
            title_bout = scraper.extract_title_bout()
            method_of_finish = scraper.extract_method_of_finish()
            details = scraper.extract_details()
            fight_details = scraper.extract_specific_fight_details()

            results.append({
                'Event Name': event_name,
                'Title Bout': title_bout,
                'Method of Finish': method_of_finish,
                'Details': details,
                'Specific Fight Details': fight_details,
            })
        else:
            print(f"Failed to load content for URL: {url}")

    return pd.DataFrame(results)


df_fight_methods = scrape_fight_methods(fight_urls)

# Display the DataFrame
print(df_fight_methods)

# Optionally, save the DataFrame to CSV
df_fight_methods.to_csv("fight_methods_and_judges_scores.csv", index=False)

                                 Event Name                Title Bout  \
0         UFC Fight Night: Magny vs. Prates         Welterweight Bout   
1         UFC Fight Night: Magny vs. Prates         Middleweight Bout   
2         UFC Fight Night: Magny vs. Prates         Bantamweight Bout   
3         UFC Fight Night: Magny vs. Prates  Women's Strawweight Bout   
4         UFC Fight Night: Magny vs. Prates         Middleweight Bout   
...                                     ...                       ...   
4161  UFC Fight Night: McDonald vs. Lineker         Middleweight Bout   
4162  UFC Fight Night: McDonald vs. Lineker  Women's Strawweight Bout   
4163  UFC Fight Night: McDonald vs. Lineker          Lightweight Bout   
4164  UFC Fight Night: McDonald vs. Lineker         Bantamweight Bout   
4165  UFC Fight Night: McDonald vs. Lineker         Middleweight Bout   

                 Method of Finish  \
0                   Method:KO/TKO   
1               Method:Submission   
2     Method

In [ ]:
# @title Judges Scorecards Scraping (methods only)

class UFCFightMethodScraper:
    def __init__(self, url):
        self.url = url
        self.soup = None

    def load_content(self):
        response = requests.get(self.url)
        if response.status_code == 200:
            self.soup = BeautifulSoup(response.content, 'html.parser')
        else:
            print(f"Failed to load content from {self.url}, status code: {response.status_code}")

    def extract_method_of_finish(self):
        # Extract method of finish (KO/TKO, Submission, Decision, etc.)
        method_tag = self.soup.find('i', class_='b-fight-details__text-item_first')
        method_of_finish = method_tag.get_text(strip=True) if method_tag else 'N/A'

        if 'Decision' in method_of_finish:
            # Only extract scores if it's a decision
            return method_of_finish, self.extract_judges_scores()
        else:
            # Return method of finish without scores if it's not a decision
            return method_of_finish, ['N/A'] * 6

    def extract_judges_scores(self):
        # Judges' scores can be located by finding text patterns like '28-29'
        score_items = self.soup.find_all('p', class_='b-fight-details__text-item')
        fighter1_scores = []
        fighter2_scores = []

        for item in score_items:
            score_text = item.get_text(strip=True)
            if '-' in score_text and 'Decision' in score_text:
                # Extract individual judge scores like '28-29'
                scores = score_text.split('-')
                if len(scores) == 2:
                    fighter1_scores.append(scores[0].strip())
                    fighter2_scores.append(scores[1].strip().replace('.', ''))

        # Ensure each fighter has 3 scores; fill with 'N/A' if any are missing
        while len(fighter1_scores) < 3:
            fighter1_scores.append('N/A')
        while len(fighter2_scores) < 3:
            fighter2_scores.append('N/A')

        return fighter1_scores + fighter2_scores

# Function to scrape multiple fight URLs
def scrape_fight_methods(fight_urls):
    results = []
    for url in fight_urls:
        scraper = UFCFightMethodScraper(url)
        scraper.load_content()

        if scraper.soup:
            method_of_finish, judges_scores = scraper.extract_method_of_finish()
            results.append({
                'Fight URL': url,
                'Method of Finish': method_of_finish,
                'Judge1_Fighter1': judges_scores[0],
                'Judge1_Fighter2': judges_scores[3],
                'Judge2_Fighter1': judges_scores[1],
                'Judge2_Fighter2': judges_scores[4],
                'Judge3_Fighter1': judges_scores[2],
                'Judge3_Fighter2': judges_scores[5]
            })
        else:
            print(f"Failed to load content for URL: {url}")

    return pd.DataFrame(results)

## Make sure fight_urls exists


df_fight_methods = scrape_fight_methods(fight_urls)

# Display the DataFrame
print(df_fight_methods)

# Optionally, save the DataFrame to CSV
df_fight_methods.to_csv("fight_methods_and_judges_scores.csv", index=False)

                                              Fight URL  \
0     http://ufcstats.com/fight-details/799acf3ae8a5...   
1     http://ufcstats.com/fight-details/18729b116144...   
2     http://ufcstats.com/fight-details/7edf2fe3a877...   
3     http://ufcstats.com/fight-details/eaa885cf7ae3...   
4     http://ufcstats.com/fight-details/6ad594777cce...   
...                                                 ...   
4162  http://ufcstats.com/fight-details/c1f74805a4ff...   
4163  http://ufcstats.com/fight-details/d1aaef0d7f0c...   
4164  http://ufcstats.com/fight-details/9d7a798f0c20...   
4165  http://ufcstats.com/fight-details/7679c5ee3514...   
4166  http://ufcstats.com/fight-details/433e3fd8782b...   

                 Method of Finish Judge1_Fighter1 Judge1_Fighter2  \
0     Method:Decision - Unanimous             N/A             N/A   
1     Method:Decision - Unanimous             N/A             N/A   
2         Method:Decision - Split             N/A             N/A   
3              

In [ ]:
# @title Fight Scraper (Model) --old but gold

class UFCFightScraper:
    def __init__(self, source, winner):
        """
        Initializes the scraper with the HTML content source and the winner's name.
        """
        self.source = source
        self.winner = winner
        self.soup = None

    def load_content(self):
        """
        Loads the HTML content from the source, which can be either a URL or a local file.
        """
        response = requests.get(self.source)
        if response.status_code == 200:
            self.soup = BeautifulSoup(response.content, 'html.parser')
        else:
            self.soup = None
            print(f"Failed to load content from {self.source}, status code: {response.status_code}")

    # def extract_judges_scores(self):
    #     """
    #     Extracts judges' score details (e.g., 27-28) from the fight if it was a decision.
    #     If the fight was not a decision, returns 'N/A' for all scores.
    #     """
    #     score_items = self.soup.find_all('p', class_='b-fight-details__text-item')
    #     fighter1_scores = []
    #     fighter2_scores = []

    #     if not score_items:
    #         # Return 'N/A' for all judges if no score details are found (non-decision fight)
    #         return ['N/A'] * 6

    #     for item in score_items:
    #         score_text = item.get_text(strip=True)
    #         if '-' in score_text:  # Format: 27-28, 30-27, etc.
    #             scores = score_text.split('-')
    #             fighter1_scores.append(scores[0].strip())  # Add Fighter 1's score
    #             fighter2_scores.append(scores[1].strip().replace('.', ''))  # Add Fighter 2's score, remove trailing dot

    #     # Ensure we have 3 scores for each fighter, fill with 'N/A' if not
    #     while len(fighter1_scores) < 3:
    #         fighter1_scores.append('N/A')
    #     while len(fighter2_scores) < 3:
    #         fighter2_scores.append('N/A')

    #     return fighter1_scores + fighter2_scores

    def extract_fight_data(self, winner_name):
        """
        Extracts fight totals and significant strikes data for all rounds and fighters.
        Adds a "Winner?" column indicating if the fighter is the winner.
        Returns two DataFrames: fight totals and significant strikes.
        """

        columns_fight_totals = [
            "Fighter", "KD", "Sig. Str.", "Sig. Str. %", "Total Str.", "TD", "TD %",
            "Sub. Att", "Rev.", "Ctrl", "Round", "Winner?"
        ]
        columns_sig_strikes = ["Fighter", "Sig. Str", "Sig. Str. %", "Head", "Body", "Leg", "Distance", "Clinch", "Ground", "Round", "Winner?"]

        round_totals_data = []
        round_sig_strikes_data = []

        # Fight totals
        current_round = 1
        fight_totals_table = self.soup.find_all('table', class_="b-fight-details__table")
        for table in fight_totals_table:
            for row in table.find_all('tr')[1:]:
                cols = row.find_all('td')
                if len(cols) == 10:  # Ensure it's a valid row for fight totals
                    fighter1_name = cols[0].find_all('p')[0].get_text(strip=True)
                    fighter2_name = cols[0].find_all('p')[1].get_text(strip=True)

                    # Determine if fighter1 or fighter2 is the winner
                    fighter1_winner = 1 if fighter1_name == winner_name else 0
                    fighter2_winner = 1 if fighter2_name == winner_name else 0


                    # Append the "Winner?" and scores to each fighter's data
                    fighter1_data = [cols[i].find_all('p')[0].get_text(strip=True) for i in range(len(cols))] + [current_round, fighter1_winner]
                    fighter2_data = [cols[i].find_all('p')[1].get_text(strip=True) for i in range(len(cols))] + [current_round, fighter2_winner]

                    fighter1_data = self.ensure_columns(fighter1_data, len(columns_fight_totals))
                    fighter2_data = self.ensure_columns(fighter2_data, len(columns_fight_totals))

                    round_totals_data.append(fighter1_data)
                    round_totals_data.append(fighter2_data)

                current_round += 1

        # Significant strikes (similar logic to fight totals)
        current_round_y = 1
        sig_strikes_table = self.soup.find_all('table', class_="b-fight-details__table")
        for table in sig_strikes_table:
            for row in table.find_all('tr')[1:]:
                cols = row.find_all('td')
                if len(cols) == 9:  # Ensure it's a valid row for significant strikes
                    fighter1_name = cols[0].find_all('p')[0].get_text(strip=True)
                    fighter2_name = cols[0].find_all('p')[1].get_text(strip=True)

                    fighter1_winner = 1 if fighter1_name == winner_name else 0
                    fighter2_winner = 1 if fighter2_name == winner_name else 0

                    fighter1_data = [cols[i].find_all('p')[0].get_text(strip=True) for i in range(len(cols))] + [current_round_y, fighter1_winner]
                    fighter2_data = [cols[i].find_all('p')[1].get_text(strip=True) for i in range(len(cols))] + [current_round_y, fighter2_winner]

                    fighter1_data = self.ensure_columns(fighter1_data, len(columns_sig_strikes))
                    fighter2_data = self.ensure_columns(fighter2_data, len(columns_sig_strikes))

                    round_sig_strikes_data.append(fighter1_data)
                    round_sig_strikes_data.append(fighter2_data)

                current_round_y += 1

        # Create DataFrames for both tables
        df_fight_totals = pd.DataFrame(round_totals_data, columns=columns_fight_totals)
        df_sig_strikes = pd.DataFrame(round_sig_strikes_data, columns=columns_sig_strikes)

        return df_fight_totals, df_sig_strikes


    def ensure_columns(self, data_row, expected_length):
        """
        Ensures that a data row has the expected number of columns.
        If columns are missing, fills them with 'N/A'.
        """
        while len(data_row) < expected_length:
            data_row.append('N/A')
        return data_row



In [ ]:
# @title Old Fight Scraper Inititializer

df_combined_fight_totals = pd.DataFrame()
df_combined_sig_strikes = pd.DataFrame()

# Loop through the fights list (which contains tuples of URLs and winners)
for event in fights:  # 'event' is a tuple containing fight links and winners
    fight_links = event[0]  # The first item in the tuple is the list of fight links
    fight_winners = event[1]  # The second item in the tuple is the list of winners

    # Iterate over each fight link and its corresponding winner
    for url, winner_name in zip(fight_links, fight_winners):
        try:
            # Initialize the scraper for each fight link and winner
            scraper = UFCFightScraper(url, winner_name)
            scraper.load_content()

            # Extract fight data
            if scraper.soup:  # Check if the soup was loaded successfully
                df_fight_totals, df_sig_strikes = scraper.extract_fight_data(winner_name)

                # Append the new data to the combined DataFrames
                df_combined_fight_totals = pd.concat([df_combined_fight_totals, df_fight_totals], ignore_index=True)
                df_combined_sig_strikes = pd.concat([df_combined_sig_strikes, df_sig_strikes], ignore_index=True)
            else:
                print(f"Failed to load content from URL: {url}")
        except Exception as e:
            print(f"Error processing URL: {url} with error: {e}")

# Save the final combined DataFrames to CSV
df_combined_fight_totals.to_csv("combined_fight_totals_with_judge_scores.csv", index=False)
df_combined_sig_strikes.to_csv("combined_significant_strikes.csv", index=False)

In [ ]:
# @title Fight Scraper (v2) -> columned rounds
import requests
from bs4 import BeautifulSoup
import pandas as pd

class UFCFightStatsScraper:
    def __init__(self, url):
        self.url = url
        self.soup = None

    def load_content(self):
        response = requests.get(self.url)
        if response.status_code == 200:
            self.soup = BeautifulSoup(response.content, 'html.parser')
        else:
            print(f"Failed to load content from {self.url}")

    def get_event(self):
        event_tag = self.soup.find('h2', class_='b-content__title')
        event_text = event_tag.get_text(strip=True)
        return event_text

    def extract_fight_details(self):
        fight_metadata = {
            "Fighter1": None, "Fighter2": None, "Winner?": None,
            "Fight Method": None, "Outcome": None, "Num Rounds": None
        }

        # Scrape metadata (winner, method, etc.)
        # Method of finish and winner extraction example (Adjust selectors as necessary)
        method_tag = self.soup.find('i', class_='b-fight-details__text-item_first')
        fight_metadata["Fight Method"] = method_tag.get_text(strip=True) if method_tag else 'N/A'

        winner_tag = self.soup.find('div', class_='b-fight-details__person')
        if winner_tag:
            fight_metadata["Winner?"] = winner_tag.get_text(strip=True)

        # Scrape rounds and stats data
        rounds_data = []
        round_tables = self.soup.find_all('table', class_="b-fight-details__table")
        for round_num, table in enumerate(round_tables[:5], start=1):  # Limit to 5 rounds
            round_data = self.extract_round_stats(table, round_num)
            rounds_data.append(round_data)

        # Pad missing rounds with null values
        while len(rounds_data) < 5:
            rounds_data.append(self.create_empty_round(round_num=len(rounds_data) + 1))

        return fight_metadata, rounds_data




    def extract_round_stats(self, table, round_num):
        """
        Extracts the statistics for each fighter in a given round from the table.
        Populates columns for strikes, significant strikes, takedowns, etc.,
        for Fighter 1 and Fighter 2, and handles missing values if a round
        does not contain all the data.
        """
        # Initialize round data with default None values for both fighters
        round_data = {
            f"Total Strike Landed F1R{round_num}": None, "Total Strike Missed F1R{round_num}": None,
            f"Non-Sig. Strike Landed F1R{round_num}": None, "Non-Sig. Strike Missed F1R{round_num}": None,
            f"Sig. Strike Landed F1R{round_num}": None, "Sig. Strike Missed F1R{round_num}": None,
            f"Head F1R{round_num}": None, "Body F1R{round_num}": None, "Leg F1R{round_num}": None,
            f"Distance F1R{round_num}": None, "Clinch F1R{round_num}": None, "Ground F1R{round_num}": None,
            f"Knockdowns F1R{round_num}": None, f"TD Completed F1R{round_num}": None,
            f"TD Missed F1R{round_num}": None, f"Sub. Att F1R{round_num}": None,
            f"Rev. F1R{round_num}": None, f"Ctrl Time (Minutes) F1R{round_num}": None,

            f"Total Strike Landed F2R{round_num}": None, "Total Strike Missed F2R{round_num}": None,
            f"Non-Sig. Strike Landed F2R{round_num}": None, "Non-Sig. Strike Missed F2R{round_num}": None,
            f"Sig. Strike Landed F2R{round_num}": None, "Sig. Strike Missed F2R{round_num}": None,
            f"Head F2R{round_num}": None, "Body F2R{round_num}": None, "Leg F2R{round_num}": None,
            f"Distance F2R{round_num}": None, "Clinch F2R{round_num}": None, "Ground F2R{round_num}": None,
            f"Knockdowns F2R{round_num}": None, f"TD Completed F2R{round_num}": None,
            f"TD Missed F2R{round_num}": None, f"Sub. Att F2R{round_num}": None,
            f"Rev. F2R{round_num}": None, f"Ctrl Time (Minutes) F2R{round_num}": None,
        }

        rows = table.find_all('tr')[1:]  # Skip header row

        # Ensure there are enough rows for both fighters
        if len(rows) < 2:
            return round_data

        # Iterate through rows for each fighter's statistics (assuming rows[0] for Fighter 1, rows[1] for Fighter 2)
        for fighter_idx, row in enumerate(rows[:2]):
            cols = row.find_all('td')

            # Set prefix for Fighter 1 or Fighter 2 based on index
            prefix = f"F{fighter_idx + 1}R{round_num}"

            # Only proceed if the row has the expected number of columns (18 columns based on the structure provided)
            if len(cols) >= 18:
                round_data[f"Total Strike Landed {prefix}"] = cols[0].get_text(strip=True)
                round_data[f"Total Strike Missed {prefix}"] = cols[1].get_text(strip=True)
                round_data[f"Non-Sig. Strike Landed {prefix}"] = cols[2].get_text(strip=True)
                round_data[f"Non-Sig. Strike Missed {prefix}"] = cols[3].get_text(strip=True)
                round_data[f"Sig. Strike Landed {prefix}"] = cols[4].get_text(strip=True)
                round_data[f"Sig. Strike Missed {prefix}"] = cols[5].get_text(strip=True)
                round_data[f"Head {prefix}"] = cols[6].get_text(strip=True)
                round_data[f"Body {prefix}"] = cols[7].get_text(strip=True)
                round_data[f"Leg {prefix}"] = cols[8].get_text(strip=True)
                round_data[f"Distance {prefix}"] = cols[9].get_text(strip=True)
                round_data[f"Clinch {prefix}"] = cols[10].get_text(strip=True)
                round_data[f"Ground {prefix}"] = cols[11].get_text(strip=True)
                round_data[f"Knockdowns {prefix}"] = cols[12].get_text(strip=True)
                round_data[f"TD Completed {prefix}"] = cols[13].get_text(strip=True)
                round_data[f"TD Missed {prefix}"] = cols[14].get_text(strip=True)
                round_data[f"Sub. Att {prefix}"] = cols[15].get_text(strip=True)
                round_data[f"Rev. {prefix}"] = cols[16].get_text(strip=True)
                round_data[f"Ctrl Time (Minutes) {prefix}"] = cols[17].get_text(strip=True)
            else:
                print(f"Warning: Row for Fighter {fighter_idx + 1} - Round {round_num} does not have enough columns")

        return round_data





    def create_empty_round(self, round_num):
        """
        Creates a dictionary with null values for all the fields in a specified round.
        This is used to pad rounds that were not fought, ensuring consistent column structure.
        """
        return {
            f"Total Strike Landed F1R{round_num}": None, f"Total Strike Missed F1R{round_num}": None,
            f"Non-Sig. Strike Landed F1R{round_num}": None, f"Non-Sig. Strike Missed F1R{round_num}": None,
            f"Sig. Strike Landed F1R{round_num}": None, f"Sig. Strike Missed F1R{round_num}": None,
            f"Head F1R{round_num}": None, f"Body F1R{round_num}": None, f"Leg F1R{round_num}": None,
            f"Distance F1R{round_num}": None, f"Clinch F1R{round_num}": None, f"Ground F1R{round_num}": None,
            f"Knockdowns F1R{round_num}": None, f"TD Completed F1R{round_num}": None,
            f"TD Missed F1R{round_num}": None, f"Sub. Att F1R{round_num}": None,
            f"Rev. F1R{round_num}": None, f"Ctrl Time (Minutes) F1R{round_num}": None,

            f"Total Strike Landed F2R{round_num}": None, f"Total Strike Missed F2R{round_num}": None,
            f"Non-Sig. Strike Landed F2R{round_num}": None, f"Non-Sig. Strike Missed F2R{round_num}": None,
            f"Sig. Strike Landed F2R{round_num}": None, f"Sig. Strike Missed F2R{round_num}": None,
            f"Head F2R{round_num}": None, f"Body F2R{round_num}": None, f"Leg F2R{round_num}": None,
            f"Distance F2R{round_num}": None, f"Clinch F2R{round_num}": None, f"Ground F2R{round_num}": None,
            f"Knockdowns F2R{round_num}": None, f"TD Completed F2R{round_num}": None,
            f"TD Missed F2R{round_num}": None, f"Sub. Att F2R{round_num}": None,
            f"Rev. F2R{round_num}": None, f"Ctrl Time (Minutes) F2R{round_num}": None
        }




def scrape_fight_stats(fight_urls):
    all_fights_data = []

    for url in fight_urls:
        scraper = UFCFightStatsScraper(url)
        scraper.load_content()

        # Get event name and fight details
        event_name = scraper.get_event()
        fight_metadata, rounds_data = scraper.extract_fight_details()

        # Combine event name, fight metadata, and rounds data
        fight_data = {"Event": event_name}  # Add event name as a new column
        fight_data.update(fight_metadata)

        # Update fight data with each round's stats
        for round_data in rounds_data:
            fight_data.update(round_data)

        all_fights_data.append(fight_data)

    # Create DataFrame with all fights
    df_fight_stats = pd.DataFrame(all_fights_data)
    return df_fight_stats

# Assume fight_urls list is set

df_fight_stats = scrape_fight_stats(fight_urls)


Streaming output truncated to the last 5000 lines.


In [ ]:
# @title Fight Scraper 9000 (Failure)


class UFCFightScraper9000():
    def __init__(self, url):
        self.url = url

        self.soup = None

    def load_content(self):
        response = requests.get(self.url)
        if response.status_code == 200:
            self.soup = BeautifulSoup(response.content, 'html.parser')
        else:
            print(f"Failed to load content from {self.url}")


    ### Fight Information ###

    def scrape_ufc_fight_details(html_content):
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Initialize a dictionary to store the fight details
        fight_details = {}

        # Extract Event details
        event_details = soup.find('h2', class_='b-content__title')
        if event_details:
            fight_details['Event'] = event_details.get_text(strip=True)

        # Extract the bout title (e.g., "UFC Featherweight Title Bout")
        bout_title = soup.find('i', class_='b-fight-details__fight-title')
        if bout_title:
            fight_details['Bout'] = bout_title.get_text(strip=True)

        # Extract fighters' names and the winner
        fighters = soup.find_all('div', class_='b-fight-details__person')
        if len(fighters) >= 2:
            fight_details["Fighter1"] = fighters[0].find('h3', class_='b-fight-details__person-name').get_text(strip=True)
            fight_details["Fighter2"] = fighters[1].find('h3', class_='b-fight-details__person-name').get_text(strip=True)

            # Identify the winner by finding the green "W" label
            winner_tag = soup.find('i', class_='b-fight-details__person-status b-fight-details__person-status_style_green')
            if winner_tag:
                winner_div = winner_tag.find_parent('div', class_='b-fight-details__person')
                if winner_div:
                    fight_details["Winner"] = winner_div.find('h3', class_='b-fight-details__person-name').get_text(strip=True)
        # Extract method, round, time, time format, and referee details
        content = soup.find('div', class_='b-fight-details__content')
        if content:
            labels = content.find_all('i', class_='b-fight-details__label')

            # Extract details if they exist
            for label in labels:
                label_text = label.get_text(strip=True)
                if label_text == "Method:":
                    fight_details['Method'] = label.find_next_sibling('i').get_text(strip=True)
                elif label_text == "Round:":
                    fight_details['Round'] = label.next_sibling.strip()
                elif label_text == "Time:":
                    fight_details['Time'] = label.next_sibling.strip()
                elif label_text == "Time format:":
                    fight_details['Time format'] = label.next_sibling.strip()
                elif label_text == "Referee:":
                    fight_details['Referee'] = label.find_next_sibling('span').get_text(strip=True)

            # Extract fight finish details if present (e.g., "Punch to Head At Distance")
            finish_details = content.find('p', class_='b-fight-details__text')
            if finish_details and "Details:" in finish_details.get_text(strip=True):
                fight_details['Details'] = finish_details.get_text(strip=True).replace("Details:", "").strip()

        return fight_details








    ####
    ######## Fight Stats (Totals) ############
    ####


    def scrape_round_statistics(html_content, rounds):

        self.html_content = html_content
        self.rounds = rounds

        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        #dynamic attempt to fetch rounds
        #rounds = scrape_ufc_fight_details.get("Round")

        # Initialize a dictionary to store the round-by-round stats
        fight_totals_stats = {}

        # Find the round tables
        round_tables = soup.find_all('table', class_='b-fight-details__table')

        for round_num in range(1, rounds + 1):
            round_header = f"Round {round_num}"

            # Find the relevant table based on round (tables are typically ordered by round)
            round_table = round_tables[round_num - 1]

            # Collect row data for each fighter (two fighters per round)
            fighter_rows = round_table.find_all('tr', class_='b-fight-details__table-row')

            for fighter_index, fighter_row in enumerate(fighter_rows, start=1):
                # Extract fighter name
                fighter_name = fighter_row.find('a', class_='b-link').get_text(strip=True)

                # Get all the stat cells for the fighter
                stat_cells = fighter_row.find_all('p', class_='b-fight-details__table-text')
                stat_labels = [
                    'KD', 'SignificantStrikes', 'SignificantStrike%', 'TotalStrikes',
                    'Takedowns', 'Takedown%', 'SubmissionAttempts', 'Reversals', 'Control'
                ]

                # Populate the stats dictionary with fighter stats for the round
                for idx, cell in enumerate(stat_cells):
                    stat_value = cell.get_text(strip=True)
                    # Create dynamic key names like F1R1KD, F1R1SignificantStrikes, etc.
                    stat_key = f"F{fighter_index}R{round_num}{stat_labels[idx]}"
                    fight_totals_stats[stat_key] = stat_value

        return fight_totals_stats




    def scrape_detailed_round_statistics(html_content, rounds):
        # Parse the HTML content with BeautifulSoup

        self.rounds = rounds
        self.html_content = html_content


        soup = BeautifulSoup(html_content, 'html.parser')


        # Initialize a dictionary to store the detailed round-by-round stats
        fight_stats = {}

        # Find the tables corresponding to the detailed stats
        # Assuming they appear in order after the main round stats table
        detailed_round_tables = soup.find_all('table', class_='b-fight-details__table js-fight-table')

        for round_num in range(1, rounds + 1):
            # Select the appropriate table for each round
            detailed_round_table = detailed_round_tables[round_num - 1]

            # Collect rows for each fighter (two fighters per round)
            fighter_rows = detailed_round_table.find_all('tr', class_='b-fight-details__table-row')

            for fighter_index, fighter_row in enumerate(fighter_rows, start=1):
                # Extract fighter name
                fighter_name = fighter_row.find('a', class_='b-link').get_text(strip=True)

                # Get all the stat cells for the fighter
                stat_cells = fighter_row.find_all('p', class_='b-fight-details__table-text')
                stat_labels = [
                    'SignificantStrikes', 'SignificantStrike%', 'HeadStrikes', 'BodyStrikes', 'LegStrikes',
                    'DistanceStrikes', 'ClinchStrikes', 'GroundStrikes'
                ]

                # Populate the stats dictionary with fighter stats for the round
                for idx, cell in enumerate(stat_cells):
                    stat_value = cell.get_text(strip=True)
                    # Create dynamic key names like F1R1HeadStrikes, F1R1BodyStrikes, etc.
                    stat_key = f"F{fighter_index}R{round_num}{stat_labels[idx]}"
                    fight_stats[stat_key] = stat_value

        return fight_stats






def main(fight_urls):
    all_fights_data = []

    for url in fight_urls:
        scraper = UFCFightScraper9000(url)

        # Scrape fight details
        fight_details = scraper.scrape_ufc_fight_details()

        # Scrape round statistics
        round_stats = scraper.scrape_round_statistics()

        # Scrape detailed round statistics
        detailed_round_stats = scraper.scrape_detailed_round_statistics()

        # Combine all data into one dictionary
        fight_data = {**fight_details, **round_stats, **detailed_round_stats}
        all_fights_data.append(fight_data)

    # Create a DataFrame from the list of all fights data
    df = pd.DataFrame(all_fights_data)

    # Export the DataFrame to a CSV file
    df.to_csv("ufc_fights_data.csv", index=False)
    print("Data exported to ufc_fights_data.csv")






In [ ]:
main(fight_urls)

TypeError: object of type 'UFCFightScraper9000' has no len()

In [ ]:
# @title (Might not be necessary)
df_combined_fight_totals = pd.DataFrame()
df_combined_sig_strikes = pd.DataFrame()

# Loop through the fights list (which contains tuples of URLs and winners)
for event in fights:  # 'event' is a tuple containing fight links and winners
    fight_links = event[0]  # The first item in the tuple is the list of fight links
    fight_winners = event[1]  # The second item in the tuple is the list of winners

    # Iterate over each fight link and its corresponding winner
    for url, winner_name in zip(fight_links, fight_winners):
        try:
            # Initialize the scraper for each fight link and winner
            scraper = UFCFightScraper(url, winner_name)
            scraper.load_content()

            # Extract fight data
            if scraper.soup:  # Check if the soup was loaded successfully
                df_fight_totals, df_sig_strikes = scraper.extract_fight_data(winner_name)

                # Append the new data to the combined DataFrames
                df_combined_fight_totals = pd.concat([df_combined_fight_totals, df_fight_totals], ignore_index=True)
                df_combined_sig_strikes = pd.concat([df_combined_sig_strikes, df_sig_strikes], ignore_index=True)
            else:
                print(f"Failed to load content from URL: {url}")
        except Exception as e:
            print(f"Error processing URL: {url} with error: {e}")

# Save the final combined DataFrames to CSV
df_combined_fight_totals.to_csv("combined_fight_totals_with_judge_scores.csv", index=False)
df_combined_sig_strikes.to_csv("combined_significant_strikes.csv", index=False)

NameError: name 'pd' is not defined